In [1]:
json_prompt = """你精通python代码的静态检查相关的工具和原理，假设我拿到了python工具bandit扫描的结果，结果是json格式的，我需要分析这个结果从而确认是否误报，是否需要高优先级修复。
首先，请根据给出的json数据，帮我找出总共有多少高危（Severity = HIGH）、中危（Severity = MEDIUM）、低危（Severity = LOW）的漏洞。

json数据：
```json
{json_data}
```
"""

**ssdlc 安全检查结果分析器**

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("./env/.env"))

import dashscope
from http import HTTPStatus
from pprint import pprint
import json

from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatTongyi

llm_model = "qwen-max"

llm = ChatTongyi(temperature=1.0, model=llm_model)

json_analysis_prompt = ChatPromptTemplate.from_template(json_prompt)

with open("./data/result.json", "r") as f:
    json_data = json.load(f)

response = llm.invoke(json_analysis_prompt.format_messages(json_data=json_data))

print(response.content)

根据你提供的JSON数据，我们可以分析出不同严重性级别的漏洞数量。这些信息在`metrics`部分的每个文件条目以及总计（`_totals`）中都有记录。具体来说，我们关注的是`SEVERITY.HIGH`, `SEVERITY.MEDIUM`, 和 `SEVERITY.LOW`这三个字段。

以下是计算结果：

1. **高危 (Severity = HIGH)**:
   - 从`_totals`中可以看到，总共有1个高危漏洞 (`SEVERITY.HIGH`: 1)。
   - 具体到文件层面，`./bench/common/system.py` 文件有1个高危漏洞。

2. **中危 (Severity = MEDIUM)**:
   - 从`_totals`中可以看到，总共有0个中危漏洞 (`SEVERITY.MEDIUM`: 0)。
   - 没有任何文件报告中危漏洞。

3. **低危 (Severity = LOW)**:
   - 从`_totals`中可以看到，总共有3个低危漏洞 (`SEVERITY.LOW`: 3)。
   - 具体到文件层面：
     - `./bench/common/system.py` 文件有1个低危漏洞。
     - `./bench/controller/benchmark.py` 文件有2个低危漏洞。

总结如下：
- 高危漏洞：1个
- 中危漏洞：0个
- 低危漏洞：3个

接下来，你可以根据这些信息来评估每个漏洞的具体情况，并决定是否需要高优先级修复。对于高危漏洞，通常建议立即处理；而对于低危漏洞，可以根据具体情况和项目需求来决定修复的优先级。
